In [3]:
import cobra
import cometspy as c
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)

# This notebook loads the four "chosen" models, discovers their growth rate on various single carbon compounds, and chooses some near the end which yield both slow and fast growth rates, and which have little variation between models.

In [4]:
model_dir = "./models/"

# 4 chosen strains: S3211.3 (3B),  S3212.3 (3F), S3212.4 (3G), S3213.3 (3I).


model_shortnames = {"B" : "ST32113_grampos",
             "F" : "ST32123_grampos",
             "G" : "ST32124_grampos",
             "I" :"ST32133_grampos"}

model_ids = list(model_shortnames.values())
B = cobra.io.read_sbml_model(model_dir + model_shortnames["B"] + "_2022gapclosed.xml")
B.id = "B"
F = cobra.io.read_sbml_model(model_dir + model_shortnames["F"] + "_2022gapclosed.xml")
F.id = "F"
G = cobra.io.read_sbml_model(model_dir + model_shortnames["G"] + "_2022gapclosed.xml")
G.id = "G"
I = cobra.io.read_sbml_model(model_dir + model_shortnames["I"] + "_2022gapclosed.xml")
I.id = "I"

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [5]:
unlimited = ["EX_ca2_e","EX_cl_e","EX_co2_e","EX_cobalt2_e","EX_cu2_e",
             "EX_fe2_e","EX_fe3_e","EX_h_e","EX_h2o_e","EX_k_e","EX_mg2_e",
             "EX_mn2_e","EX_mobd_e","EX_na1_e","EX_nh4_e","EX_ni2_e",
             "EX_o2_e","EX_pi_e","EX_sel_e","EX_slnt_e","EX_so4_e",
             "EX_tungs_e","EX_zn2_e"]

In [6]:
# find out growth rates on carbon sources
exchanges = list(set([x.id  for model in [B, F, G, I] for x in model.exchanges]))
growth = {}
for model in [B,F,G,I]:
    growth[model.id] = []
    model.medium = {key: 10 for key in unlimited if key in [x.id for x in model.exchanges]}
for ex in exchanges:
    for model in [B, F, G, I]:
        with model:
            try:
                model.exchanges.get_by_id(ex).lower_bound = -10.
                sol = model.slim_optimize(0.)
            except:
                sol = 0.
            growth[model.id].append(sol)


In [7]:
growth['exchange'] = exchanges
growth_data = pd.DataFrame(growth)
growth_data = growth_data.loc[np.logical_and(growth_data.B > 0,
                               growth_data.F > 0),].loc[np.logical_and(growth_data.G > 0,
                                                                      growth_data.I > 0),]

In [8]:
growth_data["sd"] = growth_data.drop(labels = "exchange", axis = 1).agg(np.std, axis = 1)
growth_data["mean"] = growth_data.drop(labels = ["exchange","sd"], axis = 1).agg("mean", axis = 1)
growth_data["CV"] = growth_data["sd"] / growth_data["mean"]

In [10]:
growth_data.sort_values("CV")

,B,F,G,I,exchange,sd,mean,CV
0,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_malttr_e,5.947729e-15,9.565405e-01,6.217959e-15
156,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_xylan4_e,9.104280e-15,9.565405e-01,9.517925e-15
249,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_malthx_e,1.691114e-14,9.565405e-01,1.767948e-14
19,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_lmn30_e,2.143612e-14,9.565405e-01,2.241005e-14
196,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_glucan4_e,2.903483e-14,9.565405e-01,3.035400e-14
35,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_tre_e,8.762291e-14,9.565405e-01,9.160397e-14
172,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_xyl3_e,8.560213e-13,9.565405e-01,8.949139e-13
161,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_maltttr_e,8.591926e-13,9.565405e-01,8.982292e-13
97,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_malthp_e,1.677558e-12,9.565405e-01,1.753776e-12
23,9.565405e-01,9.565405e-01,9.565405e-01,9.565405e-01,EX_glucan6_e,5.459355e-12,9.565405e-01,5.707396e-12


## This table shows us the media which minimize the variation in growth rate. From it, I will choose EX_csn_e, EX_ura_e (low growth rate) EX_but_e, EX_ad_e (med), EX_thr__L_e (med), EX_tre_e (high)

### They are not all the same growth rates within these, though they are close. I will also alter their growth rates to be exactly the same, by altering the uptake rate.